In [74]:
import pickle
import datetime

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.stats import spearmanr, kendalltau
from scipy.special import logsumexp

from sklearn.linear_model import LinearRegression

In [2]:
train_startdate = datetime.datetime.strptime('2019-01-01', "%Y-%m-%d").date()
train_enddate = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()

test_startdate = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()
test_enddate = datetime.datetime.strptime('2021-01-01', "%Y-%m-%d").date()

In [3]:
with open('./players.pkl', 'rb') as file:
    players = pickle.load(file)
    players = pd.DataFrame(players.values())

with open('./results.pkl', 'rb') as file:
    results = pickle.load(file)

with open('./tournaments.pkl', 'rb') as file:
    tournaments = pickle.load(file)
    tournaments = pd.DataFrame(tournaments.values())

In [4]:
results_df = pd.DataFrame([team_info | {"res_from": key} for key, teams in results.items() for team_info in teams])

In [5]:
notnull_mask = ~results_df['mask'].isnull()
nonnull_res = results_df[notnull_mask]
nonnull_res.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [6]:
nonnull_res.groupby(by='res_from').apply(lambda x: max(x['questionsTotal'])).sum()

136785.0

In [7]:
nonnull_res[nonnull_res['res_from'] == 22].head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [8]:
tournaments['dateStart'] = pd.to_datetime(
    tournaments['dateStart'],
    infer_datetime_format=True,
    utc = True).dt.date
tournaments['dateEnd'] = pd.to_datetime(
    tournaments['dateEnd'],
    infer_datetime_format=True,
    utc = True).dt.date

In [9]:
mask = (train_startdate <= tournaments['dateStart']) & (tournaments['dateEnd'] < train_enddate)
tournaments[mask].head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
3921,4772,Синхрон северных стран. Зимний выпуск,2019-01-05,2019-01-09,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4115,4973,Балтийский Берег. 3 игра,2019-01-25,2019-01-29,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4116,4974,Балтийский Берег. 4 игра,2019-03-01,2019-03-05,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4117,4975,Балтийский Берег. 5 игра,2019-04-05,2019-04-09,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4128,4986,ОВСЧ. 6 этап,2019-02-15,2019-02-19,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [10]:
trainset = nonnull_res[nonnull_res['res_from'].isin(tournaments[mask]['id'])]
print(len(trainset))
trainset.head()

77148


,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,res_from
366207,"{'id': 45556, 'name': 'Рабочее название', 'tow...",111111111011111110111111111100010010,"{'name': 'Рабочее название', 'town': {'id': 28...",28.0,"{'id': 56392, 'venue': {'id': 3030, 'name': 'С...",1.0,"[{'id': 91169, 'questionNumber': 15, 'answer':...",[],"[{'flag': 'Б', 'usedRating': 13507, 'rating': ...",4772
366208,"{'id': 1030, 'name': 'Сборная Бутана', 'town':...",111111111011110100101111011001011010,"{'name': 'Сборная Бутана', 'town': {'id': 346,...",25.0,"{'id': 56690, 'venue': {'id': 3151, 'name': 'У...",5.5,[],[],"[{'flag': None, 'usedRating': 13058, 'rating':...",4772
366209,"{'id': 4252, 'name': 'Ять', 'town': {'id': 197...",111111111011110101101111001011110000,"{'name': 'Ять', 'town': {'id': 197, 'name': 'М...",25.0,"{'id': 56814, 'venue': {'id': 3112, 'name': 'М...",5.5,"[{'id': 91164, 'questionNumber': 33, 'answer':...",[],"[{'flag': 'К', 'usedRating': 9584, 'rating': 9...",4772
366210,"{'id': 5444, 'name': 'Эйфью', 'town': {'id': 1...",101111101111111110001101011001111010,"{'name': 'Эйфью', 'town': {'id': 197, 'name': ...",25.0,"{'id': 56814, 'venue': {'id': 3112, 'name': 'М...",5.5,[],[],"[{'flag': 'Л', 'usedRating': 8592, 'rating': 8...",4772
366211,"{'id': 40931, 'name': 'Здоровенный Я', 'town':...",111111101011111101000111001001111110,"{'name': 'Здоровенный Я', 'town': {'id': 201, ...",25.0,"{'id': 55460, 'venue': {'id': 3117, 'name': 'М...",5.5,[],[],"[{'flag': 'Л', 'usedRating': 12069, 'rating': ...",4772


После некоторых раздумий, предлагаем следующие фичи:

    - Каждый игрок - отдельная фича
    - Каждый встреченный в трейн-тесте вопрос - отдельная фича

В качестве таргета мы рассматриваем bool - ответила команда на вопрос или нет.

При таком подходе мы можем после обучения взять веса при каждой фиче игрока и смотреть на неё, как на силу этого игрока. Отметим, что из такой модели можно вытащить сложность вопросов в трейн выборке (вес для соответствующей фичи). Однако, предсказания на трейн сете мы делать не сможем, поскольку вопросы там (скорее всего) будут другие, да и установить тот же это вопрос или нет мы не можем.

С учётом страшности наших данных (файл `results.pkl` довольно долго грузился и неслабо заполнял память), мы считаем важным оценить, сколько у нас есть игроков и вопросов, поскольку мы по сути хотим получить OHE-представления вопросов и игроков.

#### Количество вопросов:

In [11]:
num_questions_total = trainset.groupby('res_from').apply(lambda x: max(x['mask'].apply(len))).sum()
num_questions_total

31320

#### Количество игроков:

In [12]:
trainset['teamMembers'].iloc[0]

[{'flag': 'Б',
  'usedRating': 13507,
  'rating': 13507,
  'player': {'id': 6212,
   'name': 'Юрий',
   'patronymic': 'Яковлевич',
   'surname': 'Выменец'}},
 {'flag': 'Б',
  'usedRating': 10988,
  'rating': 13185,
  'player': {'id': 18332,
   'name': 'Александр',
   'patronymic': 'Витальевич',
   'surname': 'Либер'}},
 {'flag': 'Б',
  'usedRating': 8534,
  'rating': 12801,
  'player': {'id': 18036,
   'name': 'Михаил',
   'patronymic': 'Ильич',
   'surname': 'Левандовский'}},
 {'flag': 'К',
  'usedRating': 6401,
  'rating': 12801,
  'player': {'id': 22799,
   'name': 'Сергей',
   'patronymic': 'Игоревич',
   'surname': 'Николенко'}},
 {'flag': 'Б',
  'usedRating': 4252,
  'rating': 12757,
  'player': {'id': 15456,
   'name': 'Сергей',
   'patronymic': 'Владимирович',
   'surname': 'Коновалов'}},
 {'flag': 'Б',
  'usedRating': 2069,
  'rating': 12416,
  'player': {'id': 26089,
   'name': 'Ирина',
   'patronymic': 'Сергеевна',
   'surname': 'Прокофьева'}}]

In [13]:
player_ids_lst = [user_info['player']['id'] for team_info in trainset['teamMembers'] for user_info in team_info]
player_ids = {id_: idx for idx, id_ in enumerate(set(player_ids_lst))}
ids_player = {idx: id_ for id_, idx in player_ids.items()}

In [14]:
ids_player = {idx: id_ for id_, idx in player_ids.items()}

In [15]:
len(player_ids_lst), len(player_ids)

(398135, 52871)

Выходит что-то около 84000. В целом приемлемо (с учётом количества точек данных, увидим это чуть ниже).

In [16]:
trainset = trainset.assign(playerIds=[[user_info['player']['id'] for user_info in team_info] for team_info in trainset['teamMembers']])

In [17]:
train_dataset = trainset[['mask', 'res_from', 'playerIds']]
train_dataset.sample(5)

,mask,res_from,playerIds
464231,111101101011110001011101011111111011,5704,"[139405, 139406, 140827]"
389111,111100000101001011000000101100000011,4975,"[143467, 17046, 33000, 10337]"
491588,111010100101111101000000011001010011,5821,"[46412, 46409, 56366, 159213, 162362]"
460252,111000011101111001011010111001101111,5681,"[134991, 76278, 87044, 156789, 156788]"
474137,001100001100000010101100000000000000,5751,"[146533, 168877, 168882, 168878, 209534]"


In [18]:
grouped = train_dataset.groupby('res_from', group_keys=True).apply(lambda x: x[['mask', 'playerIds']])
grouped.head()

mask  \
res_from                                                
4772     366207  111111111011111110111111111100010010   
         366208  111111111011110100101111011001011010   
         366209  111111111011110101101111001011110000   
         366210  101111101111111110001101011001111010   
         366211  111111101011111101000111001001111110   

                                                  playerIds  
res_from                                                     
4772     366207   [6212, 18332, 18036, 22799, 15456, 26089]  
         366208           [1585, 40840, 1584, 10998, 16206]  
         366209  [23513, 18168, 21060, 35850, 31332, 10187]  
         366210         [36742, 28939, 54289, 15381, 27375]  
         366211  [28689, 17720, 30597, 12400, 26988, 69476]

Создадим табличку, в которой будем хранить OHE-векторы

In [89]:
ncols = len(player_ids) + num_questions_total

nrows = 0
for res_from, data in train_dataset.groupby('res_from'):
    num_questions = data['mask'].apply(len).max()
    nrows += num_questions * len(data)

X = csr_matrix((nrows, ncols))
y = []

In [90]:
X = X.tolil()

In [91]:
X

<3330443x84191 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [92]:
question_id = len(player_ids)  # questions' features begin right after players' features
for res_from, data in train_dataset.groupby('res_from'):
    # Here we have a group of teams from tournament
    num_questions = data['mask'].apply(len).max()
    
    for index, row in data.iterrows():
        # Here we have mask and ids
        
        for i, has_answered in enumerate(row['mask']):
            if has_answered not in ['0', '1']:
                continue
                
            X[len(y), question_id + i] = 1
            for id_ in row['playerIds']:
                X[len(y), player_ids[id_]] = 1
            y.append(int(has_answered))
    question_id += num_questions

In [93]:
X = X[:len(y)].tocsr()

In [94]:
lr = LinearRegression()

lr.fit(X, y)

LinearRegression()

In [95]:
player_strengths = lr.coef_[:len(player_ids)]
question_difficulties = lr.coef_[len(player_ids):]

In [96]:
player_strengths

array([ 0.04031321, -0.02253896,  0.1591458 , ...,  0.0156945 ,
        0.0795878 , -0.03485022])

Выведем Топ-20 игроков из трейн выборки

In [97]:
player_indices = [ids_player[id_] for id_ in np.argsort(player_strengths)[:-20:-1]]
players[players['id'].isin(player_indices)]

,id,name,patronymic,surname
2900,3065,Вячеслав,Андреевич,Бельков
24576,26016,Владислав,Леонидович,Пристинский
29890,31645,Александр,Евгеньевич,Тимохин
71344,81028,Илья,Сергеевич,Думанский
98885,111516,Александр,Сергеевич,Морозов
109409,122872,Ирина,Михайловна,Алябьева
125948,140824,Антон,Андреевич,Шванковский
130872,146121,Валерий,Александрович,Романёнок
133784,149272,Ирина,Алексеевна,Клубикова
149348,166151,Светлана,Евгеньевна,Жалдак


Теперь мы легко можем оценить качество команды - это некоторая (может даже нелинейная) комбинация весов игроков, которые были в трейн сете. Про игроков не из трейна мы, к сожалению, ничего сказать не можем, поэтому давайте просто не учитывать их значимость для команды (возможно, они слишком застенчивы и не участвуют в обсуждении ответов на вопрос). Попробуем просто складывать эти веса.

UPD: Мы попробовали несколько вариантов $(\exp(x), \log(x - (\min(x) - 1)))$, но они работают хуже суммы весов, поэтому давайте оставим всё как есть.

Для начала давайте преобразуем тест сет.

In [102]:
test_mask = (test_startdate <= tournaments['dateStart']) & (tournaments['dateEnd'] < test_enddate)
print(len(tournaments[test_mask]))
testset = nonnull_res[nonnull_res['res_from'].isin(tournaments[test_mask]['id'])]
print(len(testset))
testset = testset.assign(
    playerIds=[
        [
            user_info['player']['id'] for user_info in team_info
        ] for team_info in testset['teamMembers']
    ],
    teamNames=[team['name'] for team in testset['team']],
)
testset = testset.assign(
    strength=[
        sum([
            player_strengths[player_ids[id_]] for id_ in ids_ if id_ in player_ids
        ]) for ids_ in testset['playerIds']
    ]
)
test_dataset = testset[['res_from', 'playerIds', 'strength']]
test_dataset.sample(5)

406
21398


,res_from,playerIds,strength
439043,5414,"[3914, 53464, 115827, 73618, 9595]",0.310623
519540,6180,"[64434, 64435, 78335, 53129, 78336]",0.272122
465464,5708,"[190974, 71510, 71513, 94667, 221750, 222561]",0.062236
517782,6142,"[104484, 158668, 136310, 105061, 152955]",0.137936
476879,5754,"[22376, 207155, 47059, 215152]",-0.232560


Отметим, что, поскольку мы считаем силу команды - корреляцию надо смотреть с, например, $-position$ и $strength$, поскольку чем меньше позиция - тем больше сила.

In [103]:
testset[['position', 'strength']].T

,380482,380483,380484,380485,380486,380487,380488,380489,380490,380491,...,528726,528727,528728,528736,528737,528738,528739,528740,528741,528742
position,1.000000,2.000000,3.000000,4.000000,5.500000,5.500000,7.500000,7.50000,9.00000,12.000000,...,11.000000,12.000000,13.000000,1.000000,2.00000,3.000000,4.500000,4.500000,6.000000,7.000000
strength,0.612196,0.456102,0.472017,0.526465,0.436103,0.482149,0.394324,0.31469,0.52818,0.431725,...,0.101258,0.130817,-0.046687,0.205421,0.27213,0.048287,0.037014,0.179599,0.170888,-0.027189


In [100]:
def get_mean_corr(dataset, corr_func):
    return dataset.groupby('res_from').apply(lambda x: corr_func(-x['position'], x['strength'])[0]).mean()

print("Spearman correlation:", get_mean_corr(testset, spearmanr))
print("Kendall correlation:", get_mean_corr(testset, kendalltau))

Spearman correlation: 0.6790837015942941
Kendall correlation: 0.5285709674560611


Что ж, отсутствие знаний о силах некоторых игроков даёт о себе знать...

Ну и, на самом деле, линейная модель - это, конечно же, не самый лучший вариант для подобного рода задач. Иногда может быть так, что в команде большинство за неправильный ответ, когда за правильный голосует меньшинство. Наши веса, естественно, не могут в полной мере такие зависимости отображать. Впрочем, если в негативном примере из-за таких случаев уменьшит вес игрока - может, он не так уж и хорош в этой игре? Это заставляет подумать о том, что же есть "сила игрока", которую считает наша модель.

Честно говоря, пункт "предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков" нам несколько непонятен - наша модель уже делает предсказания, исходя из состава команды. Боюсь, этот этап будет опущен, поскольку он уже сделан.